In [1]:
import numpy as np

with open('data.txt') as f:
    data = f.read().split('\n')

first_blank = data.index('')
rules = data[:first_blank]
my_ticket = [int(x) for x in data[first_blank+2:first_blank+3][0].split(',')]
nearby_tickets = data[first_blank+5:]

In [2]:
rules_dict = {}
all_rules = []

for rule in rules:
    range_list = []
    for rrange in rule.split(' ')[-3:]:
        if rrange != 'or':
            min_v, max_v = rrange.split('-')
            range_list.append((int(min_v), int(max_v)))
    
    rules_dict[rule.split(':')[0]] = range_list
    all_rules.extend(range_list)

#### Part 1

In [3]:
invalid_values = []
valid_tickets = []

for ticket in nearby_tickets:
    ticket_vals = [int(x) for x in ticket.split(',')]
    ticket_valid = True
    
    for value in ticket_vals:
        value_invalid = True
        for min_v, max_v in all_rules:
            if min_v <= value <= max_v:
                value_invalid = False
                break
        
        if value_invalid:
            invalid_values.append(value)
            ticket_valid = False
            break
    
    if ticket_valid:
        valid_tickets.append(ticket_vals)

print('Ticket scanning error rate: ', sum(invalid_values))

Ticket scanning error rate:  28884


#### Part 2

In [4]:
tickets_arr = np.stack(valid_tickets)

rules_arr = list(rules_dict.keys())
rules_found = []
cols_matched = []

pos_to_rule = {}

while len(rules_found) != len(rules_arr):
    for col_idx in range(tickets_arr.shape[1]):
        if col_idx in cols_matched:
            continue
        
        val_list = tickets_arr[:, col_idx].tolist()
        matching_arrays = []

        for value in val_list:
            matching = np.zeros(len(rules_arr), dtype=np.bool)

            for rule_idx, limits in enumerate(rules_dict.values()):
                if rule_idx in rules_found:
                    continue
                
                if (limits[0][0] <= value <= limits[0][1]) or (limits[1][0] <= value <= limits[1][1]):
                    matching[rule_idx] = True
                else:
                    matching[rule_idx] = False
                
            matching_arrays.append(matching)

        matching_sum = np.array(matching_arrays).sum(axis=0)

        if np.sum(matching_sum == len(valid_tickets)) == 1:
            rule_pos = np.argmax(matching_sum)
            pos_to_rule[col_idx] = rules_arr[rule_pos]
            
            rules_found.append(rule_pos)
            cols_matched.append(col_idx)

In [5]:
departure_multiply = 1
for pos, rule in pos_to_rule.items():
    if 'departure' in rule:
        departure_multiply *= my_ticket[pos]

print(departure_multiply)

1001849322119
